# [ 5-5. 현금흐름 구현 ]

In [1]:
#pandas DataFrame 화면상 출력 한도 설정
import pandas as pd
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

## 1. 한단계씩 현금흐름 추적하기

In [2]:
from practice.astn0_overview import overview, idx, acc
from practice.astn1_sales import sales
from practice.astn2_cost import cost
from practice.astn3_equity import equity
from practice.astn4_loan import loan

In [3]:
idxno = idx[0]

#운영계좌 잔액 확인
_ = acc.oprtg.bal_strt[idxno]
acc.tmp.addamt(idxno, _, note="월초 잔액")

#자기자본 인출
_ = equity.withdraw_equity_amount(idxno, acc.oprtg)
acc.tmp.addamt(idxno, _, note="자기자본 납입")

#사업비 추정
oprtg_estmtd = cost.estimate_cost_amt(idxno)

lncst_estmtd = 0
for ln in loan.getloan(reverse=True):
    lncst_estmtd += ln.estimate_fee_amt(idxno)
    lncst_estmtd += ln.estimate_IR_amt(idxno)

cst_estmtd = oprtg_estmtd + lncst_estmtd
acc.tmp.subscd(idxno, cst_estmtd, note="추정 사업비")

#분양매출 인식
for sls in sales.dct.values():
    _ = sls.get_salesamt(acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"분양매출({sls.name})")

#대출원금 인출
for ln in loan.getloan(reverse=True):
    ln.set_loan_withdrawable(idxno)
    _ = ln.withdraw_ntnl_fixed(acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"일시대출금({ln.name})")
    
for ln in loan.getloan(reverse=True):
    _ = ln.withdraw_ntnl_flexible(acc.tmp, acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"한도대출금({ln.name})")
    
#금융비용 지출
for ln in loan.getloan(reverse=False):
    fee = ln.pay_fee_amt(acc.oprtg, idxno)
    IR = ln.pay_IR_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, fee, note=f"수수료({ln.name})")
    acc.tmp.subamt(idxno, IR, note=f"이자({ln.name})")

#일반사업비 지출
_ = cost.pay_cost_amt(acc.oprtg, idxno)
acc.tmp.subamt(idxno, _, note="일반사업비")

#대출원금 상환
for ln in loan.getloan(reverse=False):
    amtrpy = ln.repay_ntnl_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, amtrpy, note=f"대출금상환({ln.name})")
    ln.setback_loan_unwithdrawable(idxno)
    
#임시계좌 현금 조정
acc.tmp.subamt(idxno, acc.tmp.bal_end[idxno], note="임시계좌 잔액 조정")

In [4]:
acc.tmp.jnlscd

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,0,0.0,None,None,추정 사업비


In [5]:
acc.tmp.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정


In [6]:
acc.oprtg.df

,bal_strt,amt_in,amt_out,bal_end
2023-01-31,0.0,5000.0,0.0,5000.0
2023-02-28,5000.0,0.0,0.0,5000.0
2023-03-31,5000.0,0.0,0.0,5000.0
2023-04-30,5000.0,0.0,0.0,5000.0
2023-05-31,5000.0,0.0,0.0,5000.0
2023-06-30,5000.0,0.0,0.0,5000.0
2023-07-31,5000.0,0.0,0.0,5000.0
2023-08-31,5000.0,0.0,0.0,5000.0
2023-09-30,5000.0,0.0,0.0,5000.0
2023-10-31,5000.0,0.0,0.0,5000.0


In [7]:
acc.oprtg.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,보통주,None,자기자본 납입


## 2. For문을 통해 일괄적으로 현금흐름 추적하기

In [8]:
from practice.astn0_overview import overview, idx, acc
from practice.astn1_sales import sales
from practice.astn2_cost import cost
from practice.astn3_equity import equity
from practice.astn4_loan import loan

In [9]:
for idxno in idx:
    #운영계좌 잔액 확인
    _ = acc.oprtg.bal_strt[idxno]
    acc.tmp.addamt(idxno, _, note="월초 잔액")

    #자기자본 인출
    _ = equity.withdraw_equity_amount(idxno, acc.oprtg)
    acc.tmp.addamt(idxno, _, note="자기자본 납입")

    #사업비 추정
    oprtg_estmtd = cost.estimate_cost_amt(idxno)

    lncst_estmtd = 0
    for ln in loan.getloan(reverse=True):
        lncst_estmtd += ln.estimate_fee_amt(idxno)
        lncst_estmtd += ln.estimate_IR_amt(idxno)

    cst_estmtd = oprtg_estmtd + lncst_estmtd
    acc.tmp.subscd(idxno, cst_estmtd, note="추정 사업비")
    
    #분양매출 인식
    for sls in sales.dct.values():
        _ = sls.get_salesamt(acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"분양매출({sls.name})")

    #대출원금 인출
    for ln in loan.getloan(reverse=True):
        ln.set_loan_withdrawable(idxno)
        _ = ln.withdraw_ntnl_fixed(acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"일시대출금({ln.name})")

    for ln in loan.getloan(reverse=True):
        _ = ln.withdraw_ntnl_flexible(acc.tmp, acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"한도대출금({ln.name})")

    #금융비용 지출
    for ln in loan.getloan(reverse=False):
        fee = ln.pay_fee_amt(acc.oprtg, idxno)
        IR = ln.pay_IR_amt(acc.oprtg, idxno)
        acc.tmp.subamt(idxno, fee, note=f"수수료({ln.name})")
        acc.tmp.subamt(idxno, IR, note=f"이자({ln.name})")

    #일반사업비 지출
    _ = cost.pay_cost_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, _, note="일반사업비")

    #대출원금 상환
    for ln in loan.getloan(reverse=False):
        amtrpy = ln.repay_ntnl_amt(acc.oprtg, idxno)
        acc.tmp.subamt(idxno, amtrpy, note=f"대출금상환({ln.name})")
        ln.setback_loan_unwithdrawable(idxno)
        
    #자기자본 상환
    if idxno == idx[-1]: #at business end
        _ = equity.repay_equity_amt(idxno, acc.oprtg)
        acc.tmp.subamt(idxno, _, note="자기자본 상환")

    #임시계좌 현금 조정
    acc.tmp.subamt(idxno, acc.tmp.bal_end[idxno], note="임시계좌 잔액 조정")

In [10]:
acc.tmp.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정
2023-02-28,10000.0,0,None,None,월초 잔액
2023-02-28,0,10000.0,None,None,임시계좌 잔액 조정
2023-03-31,10000.0,0,None,None,월초 잔액
2023-03-31,5000,0,None,None,일시대출금(trb)
2023-03-31,5000,0,None,None,일시대출금(tra)
2023-03-31,4800.0,0,None,None,한도대출금(tra)


In [11]:
acc.oprtg.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,보통주,None,자기자본 납입
2023-01-31,5000,0,보통주,None,자기자본 납입
2023-03-31,5000,0,ntnl,None,일시대출금(trb)
2023-03-31,5000,0,ntnl,None,일시대출금(tra)
2023-03-31,4800.0,0,ntnl,None,한도대출금(tra)
...,...,...,...,...,...
2025-05-31,0,37.5,None,IR,이자(trb)
2025-05-31,0,28.888889,None,예비비,예비비
2025-05-31,0,15500.0,None,ntnl,대출금상환(tra)
2025-05-31,0,5000.0,None,ntnl,대출금상환(trb)


In [12]:
acc.oprtg.df

,bal_strt,amt_in,amt_out,bal_end
2023-01-31,0.000000,10000.0,0.000000,10000.000000
2023-02-28,10000.000000,0.0,0.000000,10000.000000
2023-03-31,10000.000000,14800.0,24763.888889,36.111111
2023-04-30,36.111111,2148.0,2125.090389,59.020722
2023-05-31,59.020722,600.0,595.161389,63.859333
2023-06-30,63.859333,600.0,663.055889,0.803444
2023-07-31,0.803444,600.0,557.898389,42.905056
2023-08-31,42.905056,600.0,625.792889,17.112167
2023-09-30,17.112167,1448.0,780.213389,684.898778
2023-10-31,684.898778,2896.0,1655.633889,1925.264889


In [13]:
sales.오피.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,1448.0,1448.0,0.0,0.0,0.0,1448.0,1448.0,-1448.0,0.0,0.0
2023-05-31,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0
2023-06-30,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0
2023-07-31,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0
2023-08-31,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0,0.0,1448.0,-1448.0,0.0,0.0
2023-09-30,0.0,0.0,1448.0,2896.0,-1448.0,0.0,0.0,1448.0,2896.0,-2896.0,0.0,0.0
2023-10-31,0.0,0.0,2896.0,5792.0,-2896.0,0.0,0.0,2896.0,5792.0,-5792.0,0.0,0.0


In [14]:
sales.근생.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-08-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
loan.tra.ntnl.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,40000.0,40000.0,0.0,0.0,0.0,9800.0,9800.0,-9800.0,0.0,30200.0
2023-04-30,0.0,0.0,0.0,40000.0,-9800.0,0.0,0.0,700.0,10500.0,-10500.0,0.0,29500.0
2023-05-31,0.0,0.0,0.0,40000.0,-10500.0,0.0,0.0,600.0,11100.0,-11100.0,0.0,28900.0
2023-06-30,0.0,0.0,0.0,40000.0,-11100.0,0.0,0.0,600.0,11700.0,-11700.0,0.0,28300.0
2023-07-31,0.0,0.0,0.0,40000.0,-11700.0,0.0,0.0,600.0,12300.0,-12300.0,0.0,27700.0
2023-08-31,0.0,0.0,0.0,40000.0,-12300.0,0.0,0.0,600.0,12900.0,-12900.0,0.0,27100.0
2023-09-30,0.0,0.0,0.0,40000.0,-12900.0,0.0,0.0,0.0,12900.0,-12900.0,0.0,27100.0
2023-10-31,0.0,0.0,0.0,40000.0,-12900.0,0.0,0.0,0.0,12900.0,-12900.0,0.0,27100.0
